In [5]:
from gurobipy import Model, GRB, quicksum, LinExpr

In [2]:
class clustering_model(object):
    def __init__(self, graph, constraints, k, gamma):
        # TODO get n_vertices
        # TODO initialize the cutset_finder

        model = Model('graph_clustering')
        
        # TODO only use model._vars
        vdict = dict()
        model._vars = []
        
        for i in range(k):
            cvars = []
            for j in range(n_vertices):
                v = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
                cvars.append(v)
                vdict[(j, i)] = v
            model._vars.append(cvars)
            
        # constraint: each vertex in exactly one cluster
        for v in range(n_vertices):
            model.addConstr(quicksum([vdict[(v, i)] for i in range(k)]), GRB.EQUAL, 1)
        
        
        obj_expr = LinExpr()
        
        # indicators for violation of cl constraints
        for (u, v, w) in constraints:
            for i in range(k):
                y = model.addVar(lb=0.0, ub=0.0, vtype=GRB.BINARY)
                model.addConstr(y >= vdict[(u, i)] + vdict[(v, i)] - 1)
                obj_expr.add(y, -w * gamma)
        
        # size of smallest cluster 
        s = model.addVar(lb=0.0, ub=n_vertices, vtype=GRB.INTEGER)
        for i in range(k):
            model.addConstr(s <= quicksum([vdict[v, i] for v in range(n_vertices)]))
        obj_expr.add(s)
        
        model.setObjective(obj_expr, GRB.MAXIMIZE)
        model._k = k
        model._relobj = None
        model._impcounter = 0
        model.Params.PreCrush = 1
        
        self.model = model
    
    def solve(self):
        pass
    